In [9]:
import pandas as pd
import gzip
import gdown
import os

file_id = "1sG69mVymv4WC9ddreESc87Pl9P6gU8Ye"
output_path = "../data/us_accidents_500k.csv"

def load_data():
    if not os.path.exists(output_path):
        gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)
    return pd.read_csv(output_path, sep=",", low_memory=False)  # Prevent dtype warnings

data = load_data()
data.columns
california_data = data[data['State'] == 'CA']
california_data

Downloading...
From (original): https://drive.google.com/uc?id=1sG69mVymv4WC9ddreESc87Pl9P6gU8Ye
From (redirected): https://drive.google.com/uc?id=1sG69mVymv4WC9ddreESc87Pl9P6gU8Ye&confirm=t&uuid=7c616167-40ce-4cf1-b243-ba8fb7c9c96f
To: /Users/santoshkumar/MSAAI/statistics/project/aai-500-02_bak/data/us_accidents_500k.csv
100%|██████████| 197M/197M [00:07<00:00, 26.6MB/s] 


,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
2,A-5006183,Source1,2,2022-08-20 13:13:00.000000000,2022-08-20 15:22:45.000000000,34.661189,-120.492822,34.661189,-120.492442,0.022,...,False,False,False,False,True,False,Day,Day,Day,Day
4,A-6690583,Source1,2,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,35.395476,-118.985995,0.046,...,False,False,False,False,False,False,Night,Night,Night,Night
13,A-4535214,Source1,2,2022-02-18 16:53:00,2022-02-18 18:59:00,32.771645,-117.161407,32.730856,-117.154680,2.845,...,False,False,False,False,False,False,Day,Day,Day,Day
21,A-1506844,Source3,2,2020-06-15 08:17:29,2020-06-15 08:47:18,34.243271,-118.424461,NaN,NaN,0.000,...,False,False,True,False,False,False,Day,Day,Day,Day
27,A-3983218,Source1,2,2022-03-22 07:22:30.000000000,2022-03-22 11:12:00.000000000,33.120965,-117.321490,33.100054,-117.308585,1.626,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499976,A-5927142,Source1,2,2021-04-10 09:09:00.000000,2021-04-10 11:10:58.000000,34.023927,-117.558418,34.023839,-117.558088,0.020,...,False,False,False,False,False,False,Day,Day,Day,Day
499986,A-6955730,Source1,2,2020-11-01 00:05:00,2020-11-01 02:26:38,33.562449,-117.672854,33.555669,-117.673064,0.469,...,False,False,False,False,False,False,Night,Night,Night,Night
499987,A-6935925,Source1,2,2020-10-01 02:54:00,2020-10-01 12:37:38,38.925281,-121.606631,38.916415,-121.605827,0.614,...,False,False,False,False,False,False,Night,Night,Night,Night
499993,A-4757020,Source1,2,2022-02-16 12:39:30,2022-02-16 17:32:00,34.065767,-118.460635,34.049130,-118.448508,1.343,...,False,False,False,False,False,False,Day,Day,Day,Day


In [11]:
# Group by city and count the number of accidents
accidents_by_city = california_data['City'].value_counts().head(10)

# Display the top 10 cities with the most accidents
print(accidents_by_city)

City
Los Angeles       10299
Sacramento         4358
San Diego          3581
San Jose           2244
Riverside          1682
Oakland            1504
Bakersfield        1465
San Bernardino     1440
Anaheim            1367
Long Beach         1352
Name: count, dtype: int64


In [12]:
# Group by county and count the number of accidents
accidents_by_county = california_data['County'].value_counts().head(10)

# Display the top 10 counties with the most accidents
print(accidents_by_county)

County
Los Angeles       34247
San Bernardino     7226
Orange             6878
San Diego          6795
Alameda            6445
Sacramento         6228
Riverside          5690
Santa Clara        4562
Contra Costa       3313
Kern               2520
Name: count, dtype: int64


In [13]:
# Filter the california_data dataframe for Alameda county
alameda_data = california_data[california_data['County'] == 'Alameda']

# Display the new dataframe
print(alameda_data)

               ID   Source  Severity                     Start_Time  \
56      A-7323211  Source1         2            2019-12-20 20:45:00   
132      A-457332  Source2         2            2017-07-01 12:23:47   
258     A-7011291  Source1         2            2020-05-05 20:41:00   
346      A-341196  Source2         3            2017-03-21 09:37:12   
395     A-2686928  Source2         4            2018-07-04 20:08:19   
...           ...      ...       ...                            ...   
499604  A-3415627  Source1         2            2016-05-03 14:04:31   
499734    A-15514  Source2         2            2016-11-07 11:22:48   
499747  A-3007615  Source2         2            2018-02-12 17:36:41   
499818  A-4496621  Source1         2  2022-05-02 18:07:30.000000000   
499902  A-3968280  Source1         2            2022-12-06 11:13:00   

                             End_Time  Start_Lat   Start_Lng    End_Lat  \
56                2019-12-20 21:18:06  37.663343 -122.105623  37.663343 

In [14]:
# Group by severity and weather condition, and count the number of accidents
severity_weather_classification = alameda_data.groupby(['Severity', 'Weather_Condition']).size().reset_index(name='Count')

# Display the classification
print(severity_weather_classification)

    Severity      Weather_Condition  Count
0          1                 Cloudy     13
1          1                   Fair     52
2          1           Fair / Windy      1
3          1             Heavy Rain      1
4          1             Light Rain      2
..       ...                    ...    ...
63         4  Mostly Cloudy / Windy      1
64         4               Overcast      2
65         4          Partly Cloudy      8
66         4       Scattered Clouds      1
67         4                  Smoke      1

[68 rows x 3 columns]
